In [20]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import os
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [21]:
config={
    'BatchSize':128,
    'seed':42,
    'n_epochs' : 50,
    'lr' : 0.001
}

seed = 42
torch.manual_seed(seed)

In [22]:

trainval_dataset = datasets.CIFAR10('../data/cifar10', train=True,download=True,transform=transforms.ToTensor())

# 前処理を定義
transform = transforms.Compose([transforms.ToTensor()])

trainval_dataset = datasets.CIFAR10('../data/cifar10', train=True, transform=transform)

# trainとvalidに分割
train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset, [len(trainval_dataset)-10000, 10000],generator=torch.Generator().manual_seed(config['seed']))

dataloader_train = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config['BatchSize'],
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=config['BatchSize'],
    shuffle=True
)

print("Train data number:{}, Valid data number: {}".format(len(train_dataset), len(val_dataset)))

Files already downloaded and verified
Train data number:40000, Valid data number: 10000


In [23]:
class NeuroGenesis1(nn.Module):
    def __init__(self,in_dim,out_dim,threshold):
        super(NeuroGenesis1,self).__init__()
        self.linear=nn.Linear(in_dim,out_dim)
        self.threshold=threshold

    def forward(self,x):
        if self.training:
            if epoch >30:
                mask=torch.abs(self.linear.weight) < self.threshold
                self.linear.weight.data[mask]=0.0

                mask=self.linear.weight==0.0
                self.linear.weight.data[mask]=torch.randn(mask.sum()).to(x.device)

                x=self.linear(x)
                return x
            else:
                return x
            
        else:
            return x

In [24]:

class NeurogenesisModel(nn.Module):
    def __init__(self):
        super(NeurogenesisModel, self).__init__()
        self.conv1=nn.Conv2d(3, 32, 3)       # 32x32x3 -> 30x30x32
        self.bn1=nn.BatchNorm2d(32)
        self.av1=nn.ReLU()
        self.pool1=nn.AvgPool2d(2)                  # 30x30x32 -> 15x15x32
        self.conv2=nn.Conv2d(32, 64, 3)             # 15x15x32 -> 13x13x64
        self.bn2=nn.BatchNorm2d(64)
        self.av2=nn.ReLU()
        self.pool2=nn.AvgPool2d(2)                  # 13x13x64 -> 6x6x64
        self.conv3=nn.Conv2d(64, 128, 3)            # 6x6x64 -> 4x4x128
        self.bn3=nn.BatchNorm2d(128)
        self.av3=nn.ReLU()
        self.pool3=nn.AvgPool2d(2)                  # 4x4x128 -> 2x2x128
        self.flatten=nn.Flatten()
        self.neurogenesis=NeuroGenesis1(2*2*128,512,0.001)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(512, 10)

        

    def forward(self, x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.av1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.bn2(x)
        x=self.av2(x)
        x=self.pool2(x)     
        x=self.conv3(x)
        x=self.bn3(x)
        x=self.av3(x)
        x=self.pool3(x)  
        x=self.flatten(x)
        x=self.neurogenesis(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x

model=NeurogenesisModel()

In [25]:

def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)

model.apply(init_weights)

device='cuda'
model.to(device)
optimizer2 = optim.Adam(model.parameters(), lr=config['lr'])
loss_function = nn.CrossEntropyLoss() 

In [26]:
for epoch in range(config['n_epochs']):
    losses_train = []
    losses_valid = []

    model.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]

        model.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = model.forward(x)  # 順伝播


        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer2.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())

    model.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = model.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]]'.format(
        epoch+1,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val,
        #{layer: getattr(model, layer).removed_neurons for layer in ['neurogenesis']}
    ))

EPOCH: 1, Train [Loss: 1.396, Accuracy: 0.498], Valid [Loss: 1.190, Accuracy: 0.578]]
EPOCH: 2, Train [Loss: 1.059, Accuracy: 0.630], Valid [Loss: 1.061, Accuracy: 0.624]]
EPOCH: 3, Train [Loss: 0.919, Accuracy: 0.680], Valid [Loss: 1.110, Accuracy: 0.618]]
EPOCH: 4, Train [Loss: 0.825, Accuracy: 0.714], Valid [Loss: 0.998, Accuracy: 0.658]]
EPOCH: 5, Train [Loss: 0.757, Accuracy: 0.739], Valid [Loss: 0.880, Accuracy: 0.693]]
EPOCH: 6, Train [Loss: 0.695, Accuracy: 0.760], Valid [Loss: 0.803, Accuracy: 0.715]]
EPOCH: 7, Train [Loss: 0.644, Accuracy: 0.780], Valid [Loss: 0.907, Accuracy: 0.691]]
EPOCH: 8, Train [Loss: 0.600, Accuracy: 0.795], Valid [Loss: 0.990, Accuracy: 0.675]]
EPOCH: 9, Train [Loss: 0.556, Accuracy: 0.809], Valid [Loss: 0.857, Accuracy: 0.704]]
EPOCH: 10, Train [Loss: 0.523, Accuracy: 0.822], Valid [Loss: 0.773, Accuracy: 0.733]]
EPOCH: 11, Train [Loss: 0.491, Accuracy: 0.834], Valid [Loss: 0.808, Accuracy: 0.727]]
EPOCH: 12, Train [Loss: 0.457, Accuracy: 0.847], Val

In [27]:
import numpy as np

def save_weights(model, filename):
    weights = {}

    # モデルの各層の名前とパラメータを取得
    for name, param in model.named_parameters():
        if 'weight' in name:
            weights[name] = param.detach().cpu().numpy()

    # .npz ファイルとして出力
    np.savez(filename, **weights)

save_weights(model, 'weights.npz')
data = np.load('weights.npz')

for key in data.keys():
    print(key, data[key].shape)

conv1.weight (32, 3, 3, 3)
bn1.weight (32,)
conv2.weight (64, 32, 3, 3)
bn2.weight (64,)
conv3.weight (128, 64, 3, 3)
bn3.weight (128,)
neurogenesis.linear.weight (512, 512)
fc2.weight (10, 512)


In [28]:
conv_net2 = nn.Sequential(
    nn.Conv2d(3, 32, 3),              # 32x32x3 -> 30x30x32
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.AvgPool2d(2),                  # 30x30x32 -> 15x15x32
    nn.Conv2d(32, 64, 3),             # 15x15x32 -> 13x13x64
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.AvgPool2d(2),                  # 13x13x64 -> 6x6x64
    nn.Conv2d(64, 128, 3),            # 6x6x64 -> 4x4x128
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.AvgPool2d(2),                  # 4x4x128 -> 2x2x128
    nn.Flatten(),
    nn.Linear(2*2*128, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)


def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)


conv_net2.apply(init_weights)

batch_size = 100
n_epochs = 5
lr = 0.01
device = 'cuda'

conv_net2.to(device)
optimizer2 = optim.Adam(conv_net2.parameters(), lr=lr)
loss_function = nn.CrossEntropyLoss()  # nn.ClossEntropyLossは，出力のsoftmax変換と，正解ラベルのone-hot vector化の機能を持っている

In [29]:
for epoch in range(config['n_epochs']):
    losses_train = []
    losses_valid = []

    conv_net2.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        n_train += t.size()[0]

        conv_net2.zero_grad()  # 勾配の初期化

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = conv_net2.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        loss.backward()  # 誤差の逆伝播

        optimizer2.step()  # パラメータの更新

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())

    conv_net2.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)

        y = conv_net2.forward(x)  # 順伝播

        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算

        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする

        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch+1,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))

EPOCH: 1, Train [Loss: 1.538, Accuracy: 0.453], Valid [Loss: 1.295, Accuracy: 0.543]
EPOCH: 2, Train [Loss: 1.057, Accuracy: 0.624], Valid [Loss: 1.142, Accuracy: 0.604]
EPOCH: 3, Train [Loss: 0.866, Accuracy: 0.697], Valid [Loss: 0.904, Accuracy: 0.683]
EPOCH: 4, Train [Loss: 0.754, Accuracy: 0.736], Valid [Loss: 0.987, Accuracy: 0.665]
EPOCH: 5, Train [Loss: 0.671, Accuracy: 0.766], Valid [Loss: 1.048, Accuracy: 0.663]
EPOCH: 6, Train [Loss: 0.589, Accuracy: 0.794], Valid [Loss: 0.924, Accuracy: 0.697]
EPOCH: 7, Train [Loss: 0.538, Accuracy: 0.813], Valid [Loss: 0.871, Accuracy: 0.713]
EPOCH: 8, Train [Loss: 0.493, Accuracy: 0.828], Valid [Loss: 0.809, Accuracy: 0.734]
EPOCH: 9, Train [Loss: 0.441, Accuracy: 0.846], Valid [Loss: 0.974, Accuracy: 0.698]
EPOCH: 10, Train [Loss: 0.398, Accuracy: 0.860], Valid [Loss: 0.894, Accuracy: 0.718]
EPOCH: 11, Train [Loss: 0.361, Accuracy: 0.874], Valid [Loss: 1.047, Accuracy: 0.683]
EPOCH: 12, Train [Loss: 0.326, Accuracy: 0.885], Valid [Loss: 0

In [30]:
import torchvision
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor() )
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)

correct = 0
total = 0
# 勾配を記憶せず（学習せずに）に計算を行う
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device),data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Files already downloaded and verified
Accuracy of the network on the 10000 test images: 40 %


In [31]:
import torchvision
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor() )
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True, num_workers=2)

correct = 0
total = 0
# 勾配を記憶せず（学習せずに）に計算を行う
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = conv_net2(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


Files already downloaded and verified
Accuracy of the network on the 10000 test images: 71 %
